First, I will look at the f3 statistic for the two anubis clusters and hamadryas

In [1]:
%run ../scripts/notebook_settings.py

In [2]:
meta_data_samples = pd.read_table("../data/metadata_with_x_missing.txt", sep=" ")

In [3]:
zarr_dir = "/faststorage/project/baboondiversity/data/PG_panu3_zarr_01_03_2021/callset.zarr/chrX"
#Opening the zarr data
callset = zarr.open_group(zarr_dir, mode="r")
gt = allel.GenotypeArray(callset["calldata/GT"])

Picking the ID values, and then extracting the allele counts.

In [4]:
meta_data_samples_f = meta_data_samples.loc[meta_data_samples.Sex=="F"]
anubis_t = meta_data_samples_f.loc[meta_data_samples_f.C_origin=="Anubis, Tanzania"].callset_index.values
anubis_e = meta_data_samples_f.loc[meta_data_samples_f.C_origin=="Anubis, Ethiopia"].callset_index.values
hamadryas = meta_data_samples_f.loc[meta_data_samples_f.C_origin=="Hamadryas, Ethiopia"].callset_index.values

In [5]:
a_t_gt = gt.take(anubis_t, axis=1)
a_e_gt = gt.take(anubis_e, axis=1)
h_gt = gt.take(hamadryas, axis=1)

In [6]:
a_t_ac = a_t_gt.count_alleles(max_allele=1)
a_e_ac = a_e_gt.count_alleles(max_allele=1)
h_ac = h_gt.count_alleles(max_allele=1)

If there is no data in one species, f3 seems to fail (division by zero), so I remove all sites which has at least one part failing.

In [8]:
passing_sites = (a_t_ac.max_allele() >= 0) & (a_e_ac.max_allele() >= 0) & (h_ac.max_allele() >= 0)

In [9]:
passing_sites.sum(), len(passing_sites)

(2911612, 2923212)

In [12]:
a_tanza = a_t_ac.compress(passing_sites)
a_ethio = a_e_ac.compress(passing_sites)
hama = h_ac.compress(passing_sites)

In [14]:
result = allel.patterson_f3(a_ethio, a_tanza, hama)

/home/eriks/miniconda3/envs/baboondiversity/lib/python3.7/site-packages/allel/stats/admixture.py:38: RuntimeWarning: invalid value encountered in true_divide
  x = (ac[:, 0] * ac[:, 1]) / (an * (an - 1))


In [20]:
result[1]

array([0.5       , 0.        , 0.        , ..., 0.        , 0.        ,
       0.23333333])